In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
exp_name = '2021_02_19_windows';
fname = f"../experiments/{exp_name}/all_data.pkl"

try:
    df_all = pd.read_pickle(fname)
    print("read", fname)
except Exception as e:
    print(e)
    print("Error: run wall_analysis.py to parse experiments.")
df_all.iloc[:, :8]

# plot_spectrograms(df_freq)

In [ ]:
from plotting_tools import plot_raw_signals, save_fig, add_colorbar
from dataset_parameters import kwargs_datasets
from pandas_utils import filter_by_dict

from wall_detector import WallDetector
from frequency_analysis import get_index_matrix

mag_thresh = 1.0 # tested for 25_* and 23_* dataset
ymin = 1e-4
ymax = 3e2
degree = 0
verbose = False

distance = df_all.iloc[0].distance
print('using unique distance', distance)

chosen_dict = {
    "degree": degree, 
    "distance": distance, 
    "mic_type": "audio_deck",
    "appendix": "_window2"
}
df_filtered = filter_by_dict(df_all, chosen_dict)
print("plotting\n", df_filtered.iloc[:, :8])

filter_by_motors = filter_by_mic_type = filter_by_appendix = False
if len(df_filtered.motors.unique()) > 1:
    filter_by_motors = True
if len(df_filtered.mic_type.unique()) > 1:
    filter_by_mic_type = True
if len(df_filtered.appendix.unique()) > 1:
    filter_by_appendix = True

for mic_type, df_mic_types in df_filtered.groupby("mic_type"):
    if mic_type == "measurement":
        mic = 0
    else:
        mic = 1
        
    for j, (i_row, row) in enumerate(df_mic_types.iterrows()):
        if (mic_type == "measurement") and (row.snr == 1):  # this is no "real snr"
            continue

        label = ""
        if filter_by_mic_type:
            label += f"{mic_type}, "
        if filter_by_motors:
            label += f"motors={row.motors}"
        if filter_by_appendix:
            label += f"{row.appendix.replace('_', ' ')} "
        label += f"mic{mic}"

        wall_detector = WallDetector.init_from_row(exp_name, row)
        spec_all = np.abs(wall_detector.current_spectrogram)
        freqs = wall_detector.current_freqs
        
        # plotting
        index_matrix = get_index_matrix(spec_all)
        spec = np.mean(spec_all, axis=1)
        
        fig, ax = plt.subplots()
        fig.set_size_inches(5, 5)
        log_spec = np.full(spec.shape, np.nan)
        log_spec[spec>0] = np.log10(spec[spec>0])
        im = ax.pcolorfast(row.seconds, freqs, log_spec[:-1, :-1])
        add_colorbar(fig, ax, im)
        ax.scatter(row.seconds[1:], 
                   freqs[index_matrix[0, :]][:-1], 
                   color='C1', ls=':', label='highest frequency', s=5)
        ax.legend()
        ax.set_xlim(
            kwargs_datasets[exp_name][row.mic_type]["min_time"],
            kwargs_datasets[exp_name][row.mic_type]["max_time"],
        )
        ax.set_ylim(
            kwargs_datasets[exp_name][row.mic_type]["min_freq"],
            kwargs_datasets[exp_name][row.mic_type]["max_freq"],
        )
        ax.set_xlabel("time [s]")
        ax.set_ylabel("frequency [Hz]")
        ax.set_title(label)
        #save_fig(
        #    fig, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_spec.png"
        #)

        fig_raw, ax = plot_raw_signals(spec_all, freqs, mic_idx=mic)
        ax.set_ylim(ymin, ymax)
        ax.set_title(label)
        save_fig(
            fig_raw, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_raw.png"
        )

## interpolation study

current status: interpolation is not helping much because we need to consider the wider shape of the peak (due to windowing) in the pipeline somewhere. The fit_peak function might be a better solution but it is also not quite working yet. 

In [ ]:
import time
from frequency_analysis import interpolate_peak, fit_peak

mic_idx = 0
chosen_dict = {
    "degree": degree, 
    "distance": distance, 
    "mic_type": "audio_deck",
    "appendix":"_window2",
    "motors":"all45000"
}
df_filtered = filter_by_dict(df_all, chosen_dict)
assert len(df_filtered) == 1
row = df_filtered.iloc[0]
wall_detector = WallDetector.init_from_row(exp_name, row)

spec = wall_detector.current_spectrogram
freqs = wall_detector.current_freqs

n_extra = 12
time_range = list(range(282, 291))
fig, ax = plt.subplots()
plot_range = list(
    range(time_range[0]-n_extra, time_range[0]) 
) + time_range + list(
    range(time_range[-1]+1, time_range[-1]+n_extra+1) 
)
spec_plot =  np.abs(spec[:, mic_idx, plot_range])
freqs_valid = np.any(spec_plot, axis=1)
spec_plot = spec_plot[freqs_valid]
im = ax.pcolorfast(plot_range, freqs[freqs_valid], spec_plot[:-1, :-1])
add_colorbar(fig, ax , im)

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)
fig.set_size_inches(10, 5)

for t_idx in time_range:
    spec_slice = spec[:, mic_idx, t_idx]

    max_without = np.max(np.abs(spec_slice))
    #print(f'magniutde estimate without fitting: {max_without:.2f}')

    # use full slice
    max_val, max_freq = interpolate_peak(spec_slice, freqs)#, ax=ax)
    
    #print(f'magniutde estimate with interpolation with all points: {max_val:.2f}')
    phase = np.angle(spec_slice)
    axs[0].plot(freqs, np.abs(spec_slice), label=t_idx)
    axs[1].plot(freqs, phase, label=t_idx)
    #ax.axvline(max_freq, color='C1')
    axs[0].set_xlim(max_freq-500, max_freq+500)
    axs[0].set_xlim(5400, 5700)

    # use slice around max
    bin_max = np.argmax(np.abs(spec_slice))
    n_points = 2
    range_max = range(bin_max-n_points, bin_max+n_points)
    spec_smaller = spec_slice[range_max]
    freq_smaller = freqs[range_max]

    max_val_smaller, max_freq_smaller = interpolate_peak(spec_smaller, freq_smaller, pad_factor=2)
    #print(f'magniutde estimate with interpolation with {n_points} points: {max_val_smaller:.2f}')
    #ax.plot(freqs, np.abs(spec_slice))
    #ax.axvline(max_freq_smaller, color='C2')
    #ax.set_xlim(max_freq_smaller-500, max_freq_smaller+500)

    max_peak, p = fit_peak(np.abs(spec_slice))
    #print(f'magniutde estimate with peak fitting: {max_peak:.2f}')
axs[0].legend()

In [ ]:
chosen_dict = {
    "degree": degree,
    "distance": distance,
    "mic_type": "audio_deck",
}
df_filtered = filter_by_dict(df_all, chosen_dict)

if mic_type == "measurement":
    mic = 0
else:
    mic = 1

n_rows = len(df_filtered.appendix.unique())
print(n_rows)
fig_slice, ax_slices = plt.subplots(n_rows, 2, sharex=True, sharey=True, squeeze=False)
fig_slice.set_size_inches(10, 5 * n_rows)

for i, (appendix, df) in enumerate(df_filtered.groupby("appendix", sort=True)):
    for j, (motors, df_row) in enumerate(df.groupby("motors", sort=True)):
        assert len(df_row) == 1, df_row.iloc[:, :10]
        row = df_row.iloc[0]
        if i == 0:
            ax_slices[0, j].set_title(motors)

        t0 = time.time()
        wall_detector = WallDetector.init_from_row(exp_name, row, interpolation='')
        t1 = time.time()
        wall_detector_lagrange = WallDetector.init_from_row(exp_name, row, interpolation='lagrange')
        t2 = time.time()
        wall_detector_quadratic = WallDetector.init_from_row(exp_name, row, interpolation='quadratic')
        t3 = time.time()
        print(f'times with interpolation:\n none:{t1-t0:.2e}s \n zero-pad:{t2-t1:.2e}s \n quadratic:{t3-t2:.2e}')

        for idx, wd in enumerate([wall_detector, wall_detector_lagrange, wall_detector_quadratic]):
            df_mic = wd.df[wd.df.mic == mic]
            label = f"interpolation {wd.interpolation}"
            stds = (
                df_mic.groupby("frequency", sort=True).magnitude.apply(np.nanstd).values
            )
            medians = (
                df_mic.groupby("frequency", sort=True)
                .magnitude.apply(np.nanmedian)
                .values
            )
            xs = np.sort(df_mic.frequency.unique())
            ax_slices[i, j].plot(xs, stds, label=label)
            
            #stds = np.r_[[np.zeros(len(medians))], [10 * stds]]
            #dx_mean = np.mean(xs[:-1]-xs[1:])
            #ax_slices[i, j].errorbar(
            #    x=xs+idx*dx_mean/5,
            #    y=medians,
            #    yerr=stds,
            #    label=label,
            #    marker="o",
            #    markersize=4.0,
            #    lw=2.0,
            #    alpha=1-idx*0.3
            #)
        ax_slices[i, j].grid(which="both")
        # sns.scatterplot(
        #    data=df_mic, x="frequency", y="magnitude", hue="counter", ax=ax, linewidth=0
        # )
        ax_slices[i, j].set_yscale('log')
    ax_slices[i, -1].legend(loc='upper right')
    title = appendix.replace("_", "")
    ax_slices[i, 0].set_ylabel(title)
    break
ax_slices[-1, 0].set_xlabel("frequency [Hz]")
ax_slices[-1, 1].set_xlabel("frequency [Hz]")
#save_fig(fig_slice, f"plots/{exp_name}_slice.png")